# Реализация свёрток

Этот ноутбук проведёт вас через процесс реализации достаточно эффективной операции свёртки.
Для иллюстрации мы будем использовать нативный numpy.

## Свёртки

Здесь мы построим базовые подходы к свёртке: от простого алгоритма с полными вложенными циклами
до варианта, который использует одно матричное умножение плюс операции изменения формы/размера.

### Порядок хранения

В простых полносвязных сетях, которые мы рассматривали до сих пор, скрытые представления обычно
хранятся как векторы, т.е. величина $z \in \mathbb{R}^n$, а для целого минибатча — как матрица
$Z \in \mathbb{R}^{B \times n}$. Но при переходе к свёрточным сетям нужно учитывать дополнительную
структуру скрытого представления. Обычно каждое скрытое состояние представляют как 3D‑массив
с размерами `height x width x channels`, а для минибатча добавляют ещё размерность батча.
То есть скрытое представление можно хранить как массив

```c++
float Z[BATCHES][HEIGHT][WIDTH][CHANNELS];
```

Этот формат называется NHWC (number(batch)‑height‑width‑channel). Однако есть и другие варианты.
Например, PyTorch по умолчанию использует формат NCHW (каналы во втором измерении, затем высота и ширина),
хотя в более поздних версиях поддерживается и NHWC. Между форматами есть тонкие, но существенные различия
по производительности: свёртки обычно быстрее в NHWC за счёт лучшего задействования tensor cores;
а формат NCHW чаще быстрее для BatchNorm (потому что в свёрточных сетях нормализация идёт по всем пикселям
одного канала).


Хотя это обсуждается реже, похожий компромисс есть и для порядка хранения весов (фильтров) свёртки.
Фильтры задаются размером ядра (теоретически он может различаться по высоте и ширине, но это редко),
числом входных каналов и числом выходных каналов. Мы будем хранить веса в виде

```c++
float weights[KERNEL_SIZE][KERNEL_SIZE][IN_CHANNELS][OUT_CHANNELS];
```

PyTorch делает иначе (так исторически сложилось): хранит веса в порядке
`OUT_CHANNELS x IN_CHANNELS x KERNEL_SIZE x KERNEL_SIZE`.

## Свёртки с простыми циклами

Начнём с простейшей реализации оператора свёртки. Мы сделаем вариант, который допускает разные размеры ядра,
но **не** содержит встроенного паддинга: чтобы добавить паддинг, нужно явно создать новый ndarray с полями.
Это означает, что если у нас входное изображение $H \times W$ и ядро размера $K$, то на выходе получится
изображение $(H - K + 1) \times (W - K + 1)$.

Хотя это в каком‑то смысле «чит», мы используем PyTorch как эталонную реализацию свёртки для проверки.
Но, поскольку PyTorch использует формат NCHW (и другой порядок весов), а мы используем NHWC и порядок,
описанный выше, нам нужно будет переставить оси перед сравнением с эталоном.

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import ctypes
from timeit import timeit
import numpy as np

def conv_reference(Z, weight):
    # NHWC -> NCHW
    Z_torch = torch.tensor(Z).permute(0,3,1,2)
    
    # KKIO -> OIKK
    W_torch = torch.tensor(weight).permute(3,2,0,1)
    
    # run convolution
    out = nn.functional.conv2d(Z_torch, W_torch)
    
    # NCHW -> NHWC
    return out.permute(0,2,3,1).contiguous().numpy()

In [51]:
np.__version__, torch.__version__

('1.26.4', '2.2.2')

In [52]:
Z = np.random.randn(10,32,32,8)
W = np.random.randn(3,3,8,16)
out = conv_reference(Z,W)
print(out.shape)

(10, 30, 30, 16)


In [53]:
%%time
out = conv_reference(Z,W)

CPU times: user 4.25 ms, sys: 1.83 ms, total: 6.08 ms
Wall time: 1.47 ms


Теперь рассмотрим самый простой вариант реализации свёртки — полностью через вложенные циклы.

In [54]:
def conv_naive(Z, weight):
    N,H,W,C_in = Z.shape
    K,_,_,C_out = weight.shape
    
    out = np.zeros((N,H-K+1,W-K+1,C_out));
    for n in range(N):
        for c_in in range(C_in):
            for c_out in range(C_out):
                for y in range(H-K+1):
                    for x in range(W-K+1):
                        for i in range(K):
                            for j in range(K):
                                out[n,y,x,c_out] += Z[n,y+i,x+j,c_in] * weight[i,j,c_in,c_out]
    return out

Мы можем проверить корректность реализации, сравнив её с эталонной версией на PyTorch.

In [55]:
out2 = conv_naive(Z,W)
print(np.linalg.norm(out - out2))

1.1988119046436182e-12


Реализация работает, но (что неудивительно: 7 вложенных циклов в коде — плохая идея)
версия на PyTorch **гораздо** быстрее; на моём ноутбуке наивная реализация медленнее в 2000-3000 раз.

In [56]:
%%time
out2 = conv_naive(Z,W)

CPU times: user 4.12 s, sys: 295 µs, total: 4.12 s
Wall time: 4.12 s


In [57]:
%%time
out = conv_reference(Z,W)

CPU times: user 4.4 ms, sys: 70 µs, total: 4.47 ms
Wall time: 1.21 ms


## Свёртки как матричные умножения

Никто не будет реализовывать свёртку поэлементно на Python. Посмотрим, как сделать намного лучше.
Простейший способ ускорения (и в целом вполне разумная реализация) — выполнить свёртку как последовательность
матричных умножений. Помните, что свёртка с ядром $K = 1$ эквивалентна матричному умножению по размерности каналов.
То есть, пусть у нас есть следующая свёртка.

In [58]:
W1 = np.random.randn(1,1,8,16)
out = conv_reference(Z,W1)
print(out.shape)

(10, 32, 32, 16)


Тогда мы можем реализовать свёртку через **одно** матричное умножение.

In [59]:
out2 = Z @ W1[0,0]
# (10, 32, 32, 8) @ (8, 16) = (10, 32, 32, 16)
print(np.linalg.norm(out - out2))

3.737906132854523e-14


In [61]:
W1[0,0].shape, out2.shape

((8, 16), (10, 32, 32, 16))

Здесь мы используем удобную особенность numpy: при умножении матриц много‑мерных массивов ведущие измерения
трактуются как набор строк матрицы. То есть операция выше эквивалентна

In [12]:
out2 = (Z.reshape(-1,8) @ W1[0,0]).reshape(Z.shape[0], Z.shape[1], Z.shape[2], W1.shape[3])

Эта идея сразу подсказывает естественный подход к свёртке: можно итерироваться только по размерам ядра $i$ и $j$,
а саму операцию свёртки выполнять через матричное умножение.

In [13]:
def conv_matrix_mult(Z, weight):
    N,H,W,C_in = Z.shape
    K,_,_,C_out = weight.shape
    out = np.zeros((N,H-K+1,W-K+1,C_out))
    
    for i in range(K):
        for j in range(K):
            out += Z[:,i:i+H-K+1,j:j+W-K+1,:] @ weight[i,j]
    return out
    

In [14]:
Z = np.random.randn(100,32,32,8)
W = np.random.randn(3,3,8,16)

out = conv_reference(Z,W)
out2 = conv_matrix_mult(Z,W)
print(np.linalg.norm(out - out2))

3.3081167894918336e-12


Это тоже работает и (как и ожидалось) **гораздо** быстрее, уже начинает конкурировать с PyTorch
(на моей машине отстаёт примерно в 2–3 раза). Увеличим размер батча, чтобы операция длилась дольше.

In [15]:
%%time
out = conv_reference(Z,W)

CPU times: user 43.9 ms, sys: 24.9 ms, total: 68.8 ms
Wall time: 8.9 ms


In [16]:
%%time
out = conv_matrix_mult(Z,W)

CPU times: user 32.4 ms, sys: 5.97 ms, total: 38.3 ms
Wall time: 37.2 ms


## Страйды

**Вопрос: как хранить матрицу в памяти?**

Строчный формат (row major): $A[i, j] => Adata[i * A.shape[1] + j]$

Колоночный формат (column major): $A[i, j] => Adata[j * A.shape[0] + i]$

Формат страйдов: $A[i, j] => Adata[i * A.strides[0] + j * A.strides[1]]$

In [62]:
def strides_in_elems(a):
    return tuple(s // a.itemsize for s in a.strides)

def get_addr(a, i, j):
    return a.ctypes.data + i * a.strides[0] + j * a.strides[1]

def read_scalar_via_pointer(a, i, j):
    addr = get_addr(a, i, j)
    if a.dtype == np.int32:
        return ctypes.c_int.from_address(addr).value
    elif a.dtype == np.float32:
        return ctypes.c_float.from_address(addr).value
    else:
        raise TypeError("Используйте int32 или float32 для демо")
    
def arr_as_one_dim(arr):
    return np.frombuffer(ctypes.string_at(arr.ctypes.data, arr.nbytes), dtype=arr.dtype, count=arr.size)

np.set_printoptions(linewidth=120)

Строчный формат и колоночный формат: данные

In [18]:
A = np.arange(1, 13, dtype=np.int32).reshape(3, 4, order='C')  # 3x4
print("A (row-major C):\n", A)
print("A.shape:", A.shape, "A.dtype:", A.dtype)
print("A.strides (bytes):", A.strides, "-> in elements:", strides_in_elems(A))
print("A.data:", arr_as_one_dim(A))

B = np.array(A, order='F', copy=True)
print("\nB (column-major F):\n", B)
print("B.strides (bytes):", B.strides, "-> in elements:", strides_in_elems(B))
print("B.data:", arr_as_one_dim(B))

A (row-major C):
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
A.shape: (3, 4) A.dtype: int32
A.strides (bytes): (16, 4) -> in elements: (4, 1)
A.data: [ 1  2  3  4  5  6  7  8  9 10 11 12]

B (column-major F):
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
B.strides (bytes): (4, 12) -> in elements: (1, 3)
B.data: [ 1  5  9  2  6 10  3  7 11  4  8 12]


Строчный формат и колоночный формат: индексирование

In [19]:
tests = [(0,0), (1,2), (2,3)]
print("Проверка индекса (i,j) -> значение у соответствующего указателя:")
for (i,j) in tests:
    vA = read_scalar_via_pointer(A, i, j)
    vB = read_scalar_via_pointer(B, i, j)
    assert A[i, j] == vA
    assert B[i, j] == vB
    assert A[i, j] == B[i, j]
    
    print(f"(i={i}, j={j})  A[i,j]={A[i,j]}  via_ptr={vA}   |   B[i,j]={B[i,j]}  via_ptr={vB}")

rows, cols = A.shape
i, j = 1, 2
c_linear = i * cols + j
f_linear = j * rows + i
print(f"\nОдномерный индекс для (i={i},j={j}):")
print("C-order  i*cols + j =", c_linear, " -> A.ravel(order='C')[..] =", A.ravel(order='C')[c_linear])
print("F-order  j*rows + i =", f_linear, " -> B.ravel(order='F')[..] =", B.ravel(order='F')[f_linear])

Проверка индекса (i,j) -> значение у соответствующего указателя:
(i=0, j=0)  A[i,j]=1  via_ptr=1   |   B[i,j]=1  via_ptr=1
(i=1, j=2)  A[i,j]=7  via_ptr=7   |   B[i,j]=7  via_ptr=7
(i=2, j=3)  A[i,j]=12  via_ptr=12   |   B[i,j]=12  via_ptr=12

Одномерный индекс для (i=1,j=2):
C-order  i*cols + j = 6  -> A.ravel(order='C')[..] = 7
F-order  j*rows + i = 7  -> B.ravel(order='F')[..] = 7


**Преимущества**: можно выполнять преобразование / слайсинг без копирования данных
- Slice: изменить начальное смещение и форму
- Transpose: поменять местами страйды
- Broadcast: вставить страйд, равный 0

In [20]:
# Slice: изменить начальное смещение и форму — всё ещё представление (view)
S = A[::2, 1:]
print("S разделяет память с A:", np.shares_memory(S, A))
print("S.strides (bytes):", S.strides, "-> в элементах:", strides_in_elems(S))

# Transpose: поменять местами страйды — всё ещё представление (view)
AT = A.T
print("\nAT = A.T")
print("AT разделяет память с A:", np.shares_memory(AT, A))
print("AT.strides (bytes):", AT.strides, "-> в элементах:", strides_in_elems(AT))

# Broadcasting: вставить страйд, равный 0 — всё ещё представление (view)
row = np.arange(A.shape[1], dtype=np.int32)[None, :]  # форма (1,4)
Bcast = np.broadcast_to(row, A.shape)                 # форма (3,4)
print("\nФорма исходной строки:", row.shape)
print("Форма представления после broadcast:", Bcast.shape)
print("Row разделяет память с Bcast:", np.shares_memory(row, Bcast))
print("Страйды после broadcast (в байтах):", Bcast.strides, "-> в элементах:", strides_in_elems(Bcast))
print("Первая строка равна второй?", np.array_equal(Bcast[0], Bcast[1]))

S разделяет память с A: True
S.strides (bytes): (32, 4) -> в элементах: (8, 1)

AT = A.T
AT разделяет память с A: True
AT.strides (bytes): (4, 16) -> в элементах: (1, 4)

Форма исходной строки: (1, 4)
Форма представления после broadcast: (3, 4)
Row разделяет память с Bcast: True
Страйды после broadcast (в байтах): (0, 4) -> в элементах: (0, 1)
Первая строка равна второй? True


**Недостатки**: доступ к памяти становится прерывистым
- Усложняет векторизацию
- Для выполнения многих операций линейной алгебры может потребоваться сначала сделать массив "непрерывным"

In [63]:
M, N = 4000, 4000
Xc = np.random.rand(M, N).astype(np.float64, copy=False)           # C-contiguous
Xf = np.array(Xc, order='F', copy=True)                            # Fortran-contiguous
XT = Xc.T                                                          # non-contiguous view

print("Xc.flags:", Xc.flags.c_contiguous, Xc.flags.f_contiguous)
print("Xf.flags:", Xf.flags.c_contiguous, Xf.flags.f_contiguous)
print("XT.flags:", XT.flags.c_contiguous, XT.flags.f_contiguous)

# Пример: суммирование по строкам у C-contiguous (строчного) массива cache-friendly;
# суммирование по строкам у транспонированного XT — нет, и наоборот.
t1 = timeit("Xc.sum(axis=1)", number=5, globals=globals())
t2 = timeit("XT.sum(axis=0)", number=5, globals=globals())
print(f"\nВремя выполнения (меньше — лучше):")
print(f"С-порядок (C-order) сумма по оси=0 (Xc): {t1:.3f}s")
print(f"Транспонированное представление (XT, не-континуальное) сумма по оси=1: {t2:.3f}s")

# Многие операции линейной алгебры требуют континуальных (неразрывных) данных;
# принудительное «упаковка» массива может улучшить производительность
t_bad  = timeit("np.dot(XT, np.ones(N, dtype=np.float32))", number=5, globals=globals())
XTc = np.ascontiguousarray(XT)  # сделать непрерывным в памяти (упаковать)
t_good = timeit("np.dot(XTc, np.ones(N, dtype=np.float32))", number=5, globals=globals())
print(f"\nУмножение dot для не-континуального XT: {t_bad:.3f}s  |  После ascontiguousarray: {t_good:.3f}s")

del Xc, Xf, XT

Xc.flags: True False
Xf.flags: False True
XT.flags: False True

Время выполнения (меньше — лучше):
С-порядок (C-order) сумма по оси=0 (Xc): 0.040s
Транспонированное представление (XT, не-континуальное) сумма по оси=1: 0.042s

Умножение dot для не-континуального XT: 0.017s  |  После ascontiguousarray: 0.016s


## Манипуляции с матрицами через «страйды»

Прежде чем реализовывать свёртки через im2col, рассмотрим пример, не связанный напрямую со свёрткой.
Вспомним эффективные операции матричного умножения из предыдущей лекции. Обычно матрицу хранят как 2D‑массив:

```c++
float A[M][N];
```

В типичном порядкe row‑major каждая из N‑мерных строк хранится подряд в памяти. Однако, чтобы лучше
использовать кэши и векторные операции современных CPU, выгодно хранить матрицу маленькими «плитками»,
чтобы векторный блок процессора эффективно работал с блоками `TILE x TILE`:

    float A[M/TILE][N/TILE][TILE][TILE];

где `TILE` — небольшая константа (например, 4). В стандартном порядке памяти для ND‑массива
такой способ укладки располагает каждый блок `TILE x TILE` подряд, что позволяет быстро
загружать/выгружать данные в кэш/регистры и т.д.

Как преобразовать матрицу к такому виду? Можно было бы вручную копировать между структурами,
но это громоздко и требует высокопроизводительного кода на C/C++. Вместо этого воспользуемся
функцией `np.lib.stride_tricks.as_strided()`, которая позволяет создавать новые представления массивов,
вручную задавая страйды, **не меняя** сами данные; затем применим `np.ascontiguousarray()`,
чтобы уложить данные последовательно. Такой приём позволяет в пару строк перестраивать матрицы довольно эффективно.

### Пример: 2D‑массив 6x6

Для наглядности возьмём пример 6x6.

In [22]:
n = 6
A = np.arange(n**2, dtype=np.float32).reshape(n,n)
print(A)

[[ 0.  1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]
 [18. 19. 20. 21. 22. 23.]
 [24. 25. 26. 27. 28. 29.]
 [30. 31. 32. 33. 34. 35.]]


Этот массив уложен в памяти построчно. Доступ к «сырой» памяти numpy намеренно затрудняет
(чтобы предотвратить ошибки), но увидеть раскладку можно так:

In [23]:
print(np.frombuffer(ctypes.string_at(A.ctypes.data, A.nbytes), dtype=A.dtype, count=A.size))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28.
 29. 30. 31. 32. 33. 34. 35.]


Ранее мы обсуждали структуру `strides` как способ описания укладки N‑мерных массивов в памяти.
Чтобы обратиться к элементу `A[i][j]` 2D‑массива, нужно взять адрес

```c++
A.bytes[i * strides[0] + j * strides[1]];
```

Аналогично для 3D‑тензора, элемент `A[i][j][k]` находится по адресу

```c++
A.bytes[i * strides[0] + j * strides[1] + k * strides[2]];
```

Для формата row‑major получим

```c++
strides[0] = num_cols;
strides[1] = 1;
```

Узнать страйды массива можно через свойство `.strides`.

In [24]:
print(A.strides)

(24, 4)


Учтите, что numpy хранит страйды в **байтах**, поэтому числа умножены на 4 (тип `float32` занимает 4 байта).


### Плиточное разбиение матрицы (matrix tiling) через страйды

Построим представление `A` в виде плиток, **только** меняя страйды. Для простоты разобьём на блоки 2x2,
т.е. хотим получить массив `3 x 3 x 2 x 2`. Какие должны быть страйды? При обращении к `A[i][j][k][l]`:
увеличение `i` сдвигает нас на две строки вниз (`strides[0] = 12`), увеличение `j` — на два столбца вправо
(`strides[1] = 2`). Далее: увеличение `k` смещает на одну строку (`strides[2] = 6`), а увеличение `l` — на один
столбец (`strides[3] = 1`).

Создадим такое представление с помощью `np.lib.stride_tricks.as_strided()`. Эта функция задаёт новую форму и
страйды **на тех же данных** (без копирования), поэтому очень эффективна — но требует осторожности,
чтобы не выйти за границы исходного массива.

In [25]:
B = np.lib.stride_tricks.as_strided(A, shape=(3,3,2,2), strides=np.array((12,2,6,1))*4)
print(B)

[[[[ 0.  1.]
   [ 6.  7.]]

  [[ 2.  3.]
   [ 8.  9.]]

  [[ 4.  5.]
   [10. 11.]]]


 [[[12. 13.]
   [18. 19.]]

  [[14. 15.]
   [20. 21.]]

  [[16. 17.]
   [22. 23.]]]


 [[[24. 25.]
   [30. 31.]]

  [[26. 27.]
   [32. 33.]]

  [[28. 29.]
   [34. 35.]]]]


Парсить вывод многомерных массивов numpy не всегда интуитивно,
но можно увидеть, что каждая подматрица 2x2 соответствует блоку исходной матрицы.
Проверим, что реальная укладка в памяти не изменилась:

In [26]:
print(np.frombuffer(ctypes.string_at(B.ctypes.data, size=B.nbytes), B.dtype, B.size))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28.
 29. 30. 31. 32. 33. 34. 35.]


In [27]:
B.strides

(48, 8, 24, 4)

Чтобы изменить порядок памяти таким образом, чтобы базовая матрица была непрерывной / компактной (что нам нужно для эффективного умножения матриц), мы можем использовать функцию `np.ascontinuugousarray()`.

In [28]:
C = np.ascontiguousarray(B)
print(np.frombuffer(ctypes.string_at(C.ctypes.data, size=C.nbytes), C.dtype, C.size))

[ 0.  1.  6.  7.  2.  3.  8.  9.  4.  5. 10. 11. 12. 13. 18. 19. 14. 15. 20. 21. 16. 17. 22. 23. 24. 25. 30. 31. 26.
 27. 32. 33. 28. 29. 34. 35.]


Как видно, массив `C` уложен в компактном порядке. Это можно подтвердить, посмотрев на `.strides`.

In [29]:
print(C.strides)
# (3, 3, 2, 2) --> (12, 4, 2, 1)

(48, 16, 8, 4)


## Свёртки через im2col

Наконец, перейдём к «настоящему» способу реализации свёрток, который по скорости будет сопоставим с PyTorch.
Идея: собрать всю вычислительную часть свёртки в **одно** матричное умножение,
чтобы воспользоваться всеми оптимизациями матричного умножения.

Ключевой приём — оператор `im2col`, который «разворачивает» 4D‑массив в вид,
удобный для умножения. Посмотрим сначала на простой 2D‑пример, а потом перейдём к 4D.
Возьмём тот же массив из начала раздела.

In [30]:
A = np.arange(36, dtype=np.float32).reshape(6,6)
print(A)

[[ 0.  1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]
 [18. 19. 20. 21. 22. 23.]
 [24. 25. 26. 27. 28. 29.]
 [30. 31. 32. 33. 34. 35.]]


И свёртку с фильтром 3x3.

In [31]:
W = np.arange(9, dtype=np.float32).reshape(3,3)
print(W)

[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]


Напомним, что свёртка умножает фильтр на каждый 3x3 блок изображения.
Как извлечь все такие блоки? Сформируем массив размера $(H - K + 1) \times (W - K + 1) \times K \times K$,
содержащий все блоки 3x3, затем сплющим его в матрицу и умножим на фильтр
Как создать такой массив без ручного копирования? Нам как раз поможет `as_strided()`.

Если создать новое представление формы `(4,4,3,3)`, какие страйды задать?
Первые два измерения будут иметь страйды 6 и 1 — как у обычного массива: шаг по первой оси — новая строка,
шаг по второй — новый столбец. «Фишка» в том, что **третье и четвёртое** измерения **тоже** имеют страйды 6 и 1,
потому что шаг по третьей оси также сдвигает нас на одну строку, а по четвёртой — на один столбец.
Посмотрим на практике.

In [32]:
B = np.lib.stride_tricks.as_strided(A, shape=(4,4,3,3), strides=4*(np.array((6,1,6,1))))
print(B)

[[[[ 0.  1.  2.]
   [ 6.  7.  8.]
   [12. 13. 14.]]

  [[ 1.  2.  3.]
   [ 7.  8.  9.]
   [13. 14. 15.]]

  [[ 2.  3.  4.]
   [ 8.  9. 10.]
   [14. 15. 16.]]

  [[ 3.  4.  5.]
   [ 9. 10. 11.]
   [15. 16. 17.]]]


 [[[ 6.  7.  8.]
   [12. 13. 14.]
   [18. 19. 20.]]

  [[ 7.  8.  9.]
   [13. 14. 15.]
   [19. 20. 21.]]

  [[ 8.  9. 10.]
   [14. 15. 16.]
   [20. 21. 22.]]

  [[ 9. 10. 11.]
   [15. 16. 17.]
   [21. 22. 23.]]]


 [[[12. 13. 14.]
   [18. 19. 20.]
   [24. 25. 26.]]

  [[13. 14. 15.]
   [19. 20. 21.]
   [25. 26. 27.]]

  [[14. 15. 16.]
   [20. 21. 22.]
   [26. 27. 28.]]

  [[15. 16. 17.]
   [21. 22. 23.]
   [27. 28. 29.]]]


 [[[18. 19. 20.]
   [24. 25. 26.]
   [30. 31. 32.]]

  [[19. 20. 21.]
   [25. 26. 27.]
   [31. 32. 33.]]

  [[20. 21. 22.]
   [26. 27. 28.]
   [32. 33. 34.]]

  [[21. 22. 23.]
   [27. 28. 29.]
   [33. 34. 35.]]]]


Это именно тот 4D‑массив, который нам нужен. Чтобы выполнить свёртку через одно матричное умножение,
 сплющим его в матрицу размера $(4\cdot4) \times (3\cdot3)$, веса — в вектор длины 9
(для многоканального случая веса станут матрицей), выполним матричное умножение и затем вернём форму 4x4.

In [33]:
print(np.frombuffer(ctypes.string_at(B.ctypes.data, size=A.nbytes), B.dtype, A.size))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28.
 29. 30. 31. 32. 33. 34. 35.]


In [34]:
B.strides

(24, 4, 24, 4)

In [35]:
C = B.reshape(16,9)
C.strides

(36, 4)

Важно отметить, что под C выделяется память!!!

In [36]:
np.shares_memory(B, C)

False

In [37]:
(B.reshape(16,9) @ W.reshape(9)).reshape(4,4)

array([[ 366.,  402.,  438.,  474.],
       [ 582.,  618.,  654.,  690.],
       [ 798.,  834.,  870.,  906.],
       [1014., 1050., 1086., 1122.]], dtype=float32)

### Важное замечание о расходе памяти

Здесь есть **очень** важный момент про эффективность по памяти.
Преобразование `W` в вектор «бесплатно» (новая память не выделяется), но преобразование `B` в 2D‑матрицу — **не бесплатное**.
Страйдовое представление `B` использует ту же память, что и `A`, но когда мы превращаем `B` в 2D‑матрицу через reshape, то уже нельзя задать её страйдами без копии — приходится **материализовывать** всю матрицу.
Это означает необходимость хранить полный im2col‑массив, что требует в $O(K^2)$ раз больше памяти
по сравнению с исходным изображением, и это может быть дорого для больших ядер.

Поэтому в современных реализациях часто **не** формируют полную im2col матрицу,
а делают «ленивое» построение или специализируют матричное умножение под im2col представления.

Все это довольно сложные темы, которые мы не будем рассматривать далее, потому что для наших целей будет достаточно просто выделить память под эту матрицу, а затем быстро освободить ее после выполнения свертки (помните, что мы, например, не выполняем backprop с помощью операции im2col).

### im2col для многоканальных свёрток

Как реализовать im2col для многоканальных, минибатч‑свёрток?
Оказывается, процесс почти такой же. Вместо 4D‑массива $(H - K + 1) \times (W - K + 1) \times K \times K$
формируем 6D‑массив $N \times (H - K + 1) \times (W - K + 1) \times K \times K \times C$
(размерности батча и каналов не трогаем). И, после небольшого обдумывания, должно быть ясно, что можно применить тот же трюк и повторить страйды
для высоты и ширины (измерения 1 и 2) в измерениях 3 и 4 (блоки $K \times K$), а страйды батча и каналов оставить.
Более того, можно просто взять существующие `.strides` входа `Z` и повторить их — без ручного подсчёта.

Чтобы посчитать свёртку, сплющим im2col‑массив в матрицу
$(N \cdot (H - K + 1) \cdot (W - K + 1)) \times (K \cdot K \cdot C)$
(помним: это неэффективно по памяти), веса — в матрицу
$(K \cdot K \cdot C) \times C_{out}$, выполним умножение и изменим форму к 4D‑выходу.
Полная реализация приведена ниже.

In [38]:
def conv_im2col(Z, weight):
    N,H,W,C_in = Z.shape
    K,_,_,C_out = weight.shape
    Ns, Hs, Ws, Cs = Z.strides
    
    inner_dim = K * K * C_in
    A = np.lib.stride_tricks.as_strided(Z, shape = (N, H-K+1, W-K+1, K, K, C_in),
                                        strides = (Ns, Hs, Ws, Hs, Ws, Cs)).reshape(-1,inner_dim)
    out = A @ weight.reshape(-1, C_out)
    return out.reshape(N,H-K+1,W-K+1,C_out)

Снова проверим совпадение с эталоном (или с предыдущими реализациями).

In [39]:
Z = np.random.randn(100,32,32,8)
W = np.random.randn(3,3,8,16)
out = conv_reference(Z,W)
out2 = conv_im2col(Z,W)
print(np.linalg.norm(out - out2))

1.6856471796294072e-12


На этом этапе скорость уже сопоставима с PyTorch: на моей машине отставание около 20-25%.

In [40]:
%%time
out3 = conv_im2col(Z,W)

CPU times: user 26.1 ms, sys: 9.16 ms, total: 35.2 ms
Wall time: 12.6 ms


In [41]:
%%time
out3 = conv_reference(Z,W)

CPU times: user 54.7 ms, sys: 68.6 ms, total: 123 ms
Wall time: 14.1 ms


## Итоговые замечания

Надеюсь, это краткое введение помогло понять, что происходит «под капотом» реализаций свёрток,
и насколько мощны манипуляции со страйдами: они позволяют выразить сложные операции без явных циклов
по матрицам (хотя часть сложности перекладывается на `.reshape` и неявный `np.ascontiguousarray()`

## ConvTranspose

Базовая иллюстрация работы транспонированной свёртки

Ниже показано, что при свёртке y и w получим обратно x с точностью до константы

In [42]:
x = torch.tensor([[[[1., 2.],
                    [3., 4.]]]])          # (1, 1, 2, 2) — вход
w = torch.tensor([[[[1., 0.],
                    [0., -1.]]]])         # (1, 1, 2, 2) — ядро

y = F.conv_transpose2d(x, w, stride=2, padding=0, output_padding=0)
print(y)

x_hat = F.conv2d(y, w, stride=2)
torch.allclose(x_hat, x * w.norm()**2)

tensor([[[[ 1.,  0.,  2.,  0.],
          [ 0., -1.,  0., -2.],
          [ 3.,  0.,  4.,  0.],
          [ 0., -3.,  0., -4.]]]])


True

Реализация транспонированной свёртки через обычную

Псевдокод:
```
x = dilate(x, stride)    # вставка нулей
x = F.pad(x, (0, output_padding, 0, output_padding))
w = weight.flip(-1,-2).permute(1,0,2,3)
y = F.conv2d(x, w, padding=K-1-padding)
```

In [43]:
def dilate(x: torch.Tensor, stride: int):
    if stride == 1:
        return x
    N, C, H, W = x.shape
    H2, W2 = (H - 1) * stride + 1, (W - 1) * stride + 1
    y = x.new_zeros(N, C, H2, W2)
    y[:, :, ::stride, ::stride] = x
    return y

class ConvTransposeViaConv(nn.Module):
    def __init__(self, in_ch, out_ch, k, stride=1, padding=0, output_padding=0, bias=True):
        super().__init__()
        self.stride, self.padding, self.output_padding = stride, padding, output_padding
        w = torch.randn(in_ch, out_ch, k, k) * (2 / (in_ch * k * k))**0.5
        self.weight = nn.Parameter(w)
        self.bias = nn.Parameter(torch.zeros(out_ch)) if bias else None

    def forward(self, x):
        x = dilate(x, self.stride)
        if self.output_padding:
            x = F.pad(x, (0, self.output_padding, 0, self.output_padding))
        w = self.weight.flip(-1, -2).permute(1, 0, 2, 3)
        p = max(self.weight.shape[-1] - 1 - self.padding, 0)
        return F.conv2d(x, w, bias=self.bias, stride=1, padding=p)

# Проверка
x = torch.randn(2, 3, 8, 8)
w = torch.randn(3, 4, 3, 3)
y_ref = F.conv_transpose2d(x, w, stride=2, padding=1, output_padding=1)
layer = ConvTransposeViaConv(3, 4, 3, stride=2, padding=1, output_padding=1)
layer.weight.data = w.clone()
y = layer(x)

print((y - y_ref).abs().max())

tensor(1.4305e-06, grad_fn=<MaxBackward1>)


Вспомогательные вычисления для семинарских заметок

In [44]:
x_tmp = torch.tensor([[[[1, 2, -3, -2],
                    [5, 1, 4, -4],
                    [0, -2, 0, -5],
                    [3, 0, -1, 1]]]])
w_tmp = torch.tensor([[[[2, 1, 4],
                    [0, 3, -5],
                    [-3, 1, -2]]]])
w_hat_tmp = torch.tensor([[[[2, 1, 4, 0, 0, 3, -5, 0, -3, 1, -2, 0, 0, 0, 0, 0],
                           [0, 2, 1, 4, 0, 0, 3, -5, 0, -3, 1, -2, 0, 0, 0, 0],
                           [0, 0, 0, 0, 2, 1, 4, 0, 0, 3, -5, 0, -3, 1, -2, 0],
                           [0, 0, 0, 0, 0, 2, 1, 4, 0, 0, 3, -5, 0, -3, 1, -2]]]])

y_tmp = F.conv2d(x_tmp, w_tmp)
z_tmp = F.conv_transpose2d(y_tmp, w_tmp, stride=1)

y_tmp, z_tmp

(tensor([[[[-27,  41],
           [ 14,  12]]]]),
 tensor([[[[ -54,   55,  -67,  164],
           [  28,  -43,  326, -157],
           [  81, -108,   61, -142],
           [ -42,  -22,  -16,  -24]]]]))

In [45]:
torch.allclose(w_hat_tmp @ x_tmp.flatten(), y_tmp.flatten())

True

In [46]:
torch.allclose(w_hat_tmp.transpose(-1, -2) @ y_tmp.flatten(), z_tmp.flatten())

True